In [1]:
import os
import sys
import re
import json

import pandas as pd
import numpy as np
from itertools import product

from tqdm.auto import tqdm

tqdm.pandas()

In [2]:
os.makedirs("data/src/", exist_ok=True)

# dataframe

## generate fake data

In [3]:
df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
  )
  

df2 = pd.DataFrame(
    {
        "A": ["A4", "A5", "A6", "A7"],
        "B": ["B4", "B5", "B6", "B7"],
        "C": ["C4", "C5", "C6", "C7"],
        "D": ["D4", "D5", "D6", "D7"],
    },
    index=[4, 5, 6, 7],
  )
  

df3 = pd.DataFrame(
    {
        "A": ["A8", "A9", "A10", "A11"],
        "B": ["B8", "B9", "B10", "B11"],
        "C": ["C8", "C9", "C10", "C11"],
        "D": ["D8", "D9", "D10", "D11"],
    },
    index=[8, 9, 10, 11],
  )

df1.to_parquet("data/src/df1.parquet")
df2.to_parquet("data/src/df2.parquet")
df3.to_parquet("data/src/df3.parquet")

## load data

In [4]:
df1 = pd.read_parquet("data/src/df1.parquet")
df2 = pd.read_parquet("data/src/df2.parquet")
df3 = pd.read_parquet("data/src/df3.parquet")
# pd.read_csv("test.csv", sep="\t", names=["a" "b", "c"])
# pd.read_excel
# pd.read_sql
print(df1)
print(df2)
print(df3)

    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1
2  A2  B2  C2  D2
3  A3  B3  C3  D3
    A   B   C   D
4  A4  B4  C4  D4
5  A5  B5  C5  D5
6  A6  B6  C6  D6
7  A7  B7  C7  D7
      A    B    C    D
8    A8   B8   C8   D8
9    A9   B9   C9   D9
10  A10  B10  C10  D10
11  A11  B11  C11  D11


In [5]:
# load dirs
df = pd.read_parquet("data/src")
df

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7
8,A8,B8,C8,D8
9,A9,B9,C9,D9


In [6]:
# add column
df["E"] = df.D.progress_apply(lambda x: f"{x}_E")
df

  0%|          | 0/12 [00:00<?, ?it/s]

,A,B,C,D,E
0,A0,B0,C0,D0,D0_E
1,A1,B1,C1,D1,D1_E
2,A2,B2,C2,D2,D2_E
3,A3,B3,C3,D3,D3_E
4,A4,B4,C4,D4,D4_E
5,A5,B5,C5,D5,D5_E
6,A6,B6,C6,D6,D6_E
7,A7,B7,C7,D7,D7_E
8,A8,B8,C8,D8,D8_E
9,A9,B9,C9,D9,D9_E


In [7]:
df.loc[1:5, "A"] = np.nan
df

,A,B,C,D,E
0,A0,B0,C0,D0,D0_E
1,NaN,B1,C1,D1,D1_E
2,NaN,B2,C2,D2,D2_E
3,NaN,B3,C3,D3,D3_E
4,NaN,B4,C4,D4,D4_E
5,NaN,B5,C5,D5,D5_E
6,A6,B6,C6,D6,D6_E
7,A7,B7,C7,D7,D7_E
8,A8,B8,C8,D8,D8_E
9,A9,B9,C9,D9,D9_E


## query dataframe

In [8]:
uplimit = "C8"
df.query("A.isna() | (C > 'C2' & C < @uplimit)", engine="python")

,A,B,C,D,E
1,NaN,B1,C1,D1,D1_E
2,NaN,B2,C2,D2,D2_E
3,NaN,B3,C3,D3,D3_E
4,NaN,B4,C4,D4,D4_E
5,NaN,B5,C5,D5,D5_E
6,A6,B6,C6,D6,D6_E
7,A7,B7,C7,D7,D7_E


## Merge, join, concatenate and compare
- https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
<img src="img/sql_join.png" alt="drawing" style="width:600px;"/>

- merge 自由度高，自定义左右 DataFrame 的 key， left_on|left_index，简单 validate merge 结果；
- join 简化版 merge，参数量少，只能基于 index join
- concat: Concatenate pandas objects along a particular axis with optional set logic along the other axes.
- append: shortcut to concat

In [9]:
df2.reset_index(inplace=True, drop=True)
df3.index = [2, 3, 4, 5]
pd.concat([df1, df2, df3])

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
0,A4,B4,C4,D4
1,A5,B5,C5,D5
2,A6,B6,C6,D6
3,A7,B7,C7,D7
2,A8,B8,C8,D8
3,A9,B9,C9,D9


In [10]:
# align index
pd.concat([df2, df3], axis=1)

,A,B,C,D,A,B,C,D
0,A4,B4,C4,D4,NaN,NaN,NaN,NaN
1,A5,B5,C5,D5,NaN,NaN,NaN,NaN
2,A6,B6,C6,D6,A8,B8,C8,D8
3,A7,B7,C7,D7,A9,B9,C9,D9
4,NaN,NaN,NaN,NaN,A10,B10,C10,D10
5,NaN,NaN,NaN,NaN,A11,B11,C11,D11


In [11]:
pd.merge(
    df2,
    df3,
    how="inner", # 'left', 'right', 'outer', 'inner'
    on=None,
    left_on=None,
    right_on=None,
    left_index=True,
    right_index=True,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,

)

,A_x,B_x,C_x,D_x,A_y,B_y,C_y,D_y
2,A6,B6,C6,D6,A8,B8,C8,D8
3,A7,B7,C7,D7,A9,B9,C9,D9


In [12]:
df2.append(df3)

,A,B,C,D
0,A4,B4,C4,D4
1,A5,B5,C5,D5
2,A6,B6,C6,D6
3,A7,B7,C7,D7
2,A8,B8,C8,D8
3,A9,B9,C9,D9
4,A10,B10,C10,D10
5,A11,B11,C11,D11


## group by and rank

In [13]:
df_group = pd.DataFrame(
    [[1, 4, 3, 2],
     [1, 5, 3, 2],
     [1, 1, 3, 2],
     [1, 3, 3, 2],
     [2, 1, 3, 2],
     [2, 3, 3, 2],
     [2, 2, 3, 2],
     [2, 2, 3, 2]],
    columns=["a", "b", "c", "d"]
)

df_group

,a,b,c,d
0,1,4,3,2
1,1,5,3,2
2,1,1,3,2
3,1,3,3,2
4,2,1,3,2
5,2,3,3,2
6,2,2,3,2
7,2,2,3,2


In [14]:
df_group.groupby("a", as_index=False) \
  .apply(lambda x: x.nlargest(2, "b")) \
  .reset_index(drop=True)

,a,b,c,d
0,1,5,3,2
1,1,4,3,2
2,2,3,3,2
3,2,2,3,2


In [15]:
# method : {'average', 'min', 'max', 'first', 'dense'}, default 'average'
#     * average: average rank of group.
#     * min: lowest rank in group.
#     * max: highest rank in group.
#     * first: ranks assigned in order they appear in the array.
#     * dense: like 'min', but rank always increases by 1 between groups.
df_group["rank"] = df_group.groupby("a")["b"].rank("dense", ascending=False)

df_group

,a,b,c,d,rank
0,1,4,3,2,2.0
1,1,5,3,2,1.0
2,1,1,3,2,4.0
3,1,3,3,2,3.0
4,2,1,3,2,3.0
5,2,3,3,2,1.0
6,2,2,3,2,2.0
7,2,2,3,2,2.0


In [16]:
# df_group.sort_values(["a", "b"], ascending=[True, False])
df_group.sort_values(["a", "b"], ascending=False)

,a,b,c,d,rank
5,2,3,3,2,1.0
6,2,2,3,2,2.0
7,2,2,3,2,2.0
4,2,1,3,2,3.0
1,1,5,3,2,1.0
0,1,4,3,2,2.0
3,1,3,3,2,3.0
2,1,1,3,2,4.0


In [17]:
# accumulation after sorted
df_group.sort_values(["a", "b"], ascending=False).expanding(1).count()

,a,b,c,d,rank
5,1.0,1.0,1.0,1.0,1.0
6,2.0,2.0,2.0,2.0,2.0
7,3.0,3.0,3.0,3.0,3.0
4,4.0,4.0,4.0,4.0,4.0
1,5.0,5.0,5.0,5.0,5.0
0,6.0,6.0,6.0,6.0,6.0
3,7.0,7.0,7.0,7.0,7.0
2,8.0,8.0,8.0,8.0,8.0


## transform

### cell of array to multi-rows

In [18]:
df_list = df_group.groupby("a", as_index=False).agg({"b": list, "c": "count", "d": ["sum", list, "count"]})
df_list

a             b     c   d                    
              list count sum          list count
0  1  [4, 5, 1, 3]     4   8  [2, 2, 2, 2]     4
1  2  [1, 3, 2, 2]     4   8  [2, 2, 2, 2]     4

In [19]:
df_list.columns

MultiIndex([('a',      ''),
            ('b',  'list'),
            ('c', 'count'),
            ('d',   'sum'),
            ('d',  'list'),
            ('d', 'count')],
           )

In [20]:
# explode cell: array to row
df_list.explode(column=(('b',  'list')))

a    b     c   d                    
     list count sum          list count
0  1    4     4   8  [2, 2, 2, 2]     4
0  1    5     4   8  [2, 2, 2, 2]     4
0  1    1     4   8  [2, 2, 2, 2]     4
0  1    3     4   8  [2, 2, 2, 2]     4
1  2    1     4   8  [2, 2, 2, 2]     4
1  2    3     4   8  [2, 2, 2, 2]     4
1  2    2     4   8  [2, 2, 2, 2]     4
1  2    2     4   8  [2, 2, 2, 2]     4

### cell of array to multi-columns

In [21]:
df_tmp = df_list.copy()
new_columns = ["b_1", "b_2", "b_3", "b_4"]
df_tmp.set_index("a", inplace=True)
df_tmp[new_columns] = pd.DataFrame(
    df_tmp[("b", "list")].to_list(), 
    columns=new_columns, 
    index=df_tmp.index  # set index required
)
df_tmp

b     c   d                     b_1 b_2 b_3 b_4
           list count sum          list count                
a                                                            
1  [4, 5, 1, 3]     4   8  [2, 2, 2, 2]     4   4   5   1   3
2  [1, 3, 2, 2]     4   8  [2, 2, 2, 2]     4   1   3   2   2

In [22]:
df_tmp = df_list.copy()
new_columns = ["b_1", "b_2", "b_3", "b_4"]
df_tmp.set_index("a", inplace=True)
df_tmp[new_columns] = pd.DataFrame(
    df_tmp[("b", "list")].to_list(), 
    columns=new_columns, 
    # index=df_list.index  # without index
)
df_tmp

b     c   d                      b_1  b_2  b_3  b_4
           list count sum          list count                    
a                                                                
1  [4, 5, 1, 3]     4   8  [2, 2, 2, 2]     4  1.0  3.0  2.0  2.0
2  [1, 3, 2, 2]     4   8  [2, 2, 2, 2]     4  NaN  NaN  NaN  NaN

## Multi-Index

In [23]:
# requirements:
# $ pip install pandas pyarrow openpyxl
#   - pandas: dataframe
#   - pyarrow: read parquet file
#   - openpyxl: process excel
#
 
import pandas as pd
 
# create sample
df = pd.DataFrame(
  [[1, 2, 3, 4, 5],
   [1, 2, 3, 4, 6],
   [1, 2, 4, 5, 7],
   [2, 3, 4, 5, 6],
   [2, 3, 4, 7, 9]],
  columns=["a", "b", "c", "d", "e"]
)

 
index_cols = ["a", "b", "c", "d"]
# df.set_index(index_cols).to_excel("test.xlsx", index_label=index_cols)
df.set_index(index_cols)

e
a b c d   
1 2 3 4  5
      4  6
    4 5  7
2 3 4 5  6
      7  9

in excel:

<img src="img/multi_index.png" alt="drawing" style="width:300px;"/>